<img src="Figs/Banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> Offset Estimation <b> </font>

<font size="5"> <b> The *ampcor* program<font color='rgba(200,0,0,0.2)'> </font> </b> </font>

<br> <img src="Figs/NASALogo.png" width="250" align="right" /> <br> 
<font size="4"> <b> Paul A Rosen</b> 
<font size="3">  <br>
<font> <b>Date: </b> July 10, 2020 </font>
</font>


In [25]:
import warnings
warnings.filterwarnings('ignore')

bShowInline = True  # Set = False for document generation
%matplotlib inline
import matplotlib.pyplot as plt

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

def makeplot( plt, figlabel, figcaption):
    figname = figlabel+'.png'

    plt.savefig(figname)

    if bShowInline:
        plt.show()
    else:
        plt.close()

    strLatex="""
    \\begin{figure}[b]
    \centering
        \includegraphics[totalheight=10.0cm]{%s}
        \caption{%s}
        \label{fig:%s}
    \end{figure}"""%(figname, figcaption, figlabel) 
    return display(Latex(strLatex)) 

def sinc_interp(x, s, u):
    # x is the vector to be interpolated
    # s is a vector of sample points of x
    # u is a vector of the output sample points for the interpolation
    
    if len(x) != len(s):
        raise ValueError('x and s must be the same length')
    
    # Find the period    
    T = s[1] - s[0]
    
    sincM = np.tile(u, (len(s), 1)) - np.tile(s[:, np.newaxis], (1, len(u)))
    y = np.dot(x, np.sinc(sincM/T))
    return y

# Overview

In this notebook, we will describe the essential algorithms in the *ampcor* program, which is used to estimate the misregistration between two images. The basic methodology uses image chip cross-correlation in the image spatial domain.  A reference image chip with a particular dimension is defined in the first image, as well as a search image chip in a second image. The search image is larger than the reference image so it can be translated in two dimensions for cross-correlation.  This is done once at nominal image sampling to establish a coarse offset.  Once the coarse offset is found, the images are oversampled by a factor of 2 and the cross-correlation is recomputed.   The correlation surface is stored, and oversampled by a factor of 32, then the peak location is estimated by curve-fitting.  The peak location is then recorded as the offset.  Much of the program is book-keeping and oversampling.  The basic functionality is just two-dimensional cross correlation.

In this notebook, we go through the ampcor program step by step, and emulate the operations with computational python and with illustrations to ensure the algorithm and methods are clear.

1.0 [Setup](#section-1)<br>
> 1.1 [Sinc interpolator](#section-1.1) <br>
> 1.2 [Input data types](#section-1.2) <br>
> 1.3 [Bounds Checking on Input](#section-1.3) <br>
> 1.4 [The Ampcor Workflow](#section-1.4) <br>


2.0 [Read in the data](#section-2)  <br>
> 2.1 [Complex Images](#section-2.1) <br>
> 2.2 [Real Images](#section-2.2) <br>
> 2.3 [Mag Images](#section-2.3) <br>

3.0 [Focusing SAR data - Range](#section-3) <br>
> 3.1 [Correlation to achieve fine range resolution - time domain](#section-3.1) <br>
> 3.2 [Correlation to achieve fine range resolution - frequency domain](#section-3.2) 

4.0 [Focusing SAR data - Azimuth](#section-4)  <br>
> 4.1 [Azimuth reference function](#section-4.2)  <br>
> 4.2 [Correlation to achieve fine azimuth resolution - time domain](#section-4.2) <br>
> 4.3 [Correlation to achieve fine azimuth resolution - frequency domain](#section-4.3) <br>
> 4.4 [Backprojection](#section-4.4)

<a id="section-1"></a>
## 1.0 Setup

<a id="section-1.1"></a>
### 1.1 Sinc Interpolator 

Ampcor defines its own sinc interpolator function, which is designed to be fast on a cpu by precomputing a very finely sampled function (oversampled by 4K or 8K), and accessing the appropriate coefficients for a particular interpolating fraction by simply selecting the nearest appropriate offset.  The sinc is obviously truncated, and an important feature of this function is the ability to taper the amplitude of the function for a smooth transition to zero at the edge.  This helps preserve phase independent of the fraction of the pixel selected. 

#### Filter definition

The elements of the function are 

| Parameter | Symbol | Variable | Ampcor Value |
| --- | --- | --- | --- |
| Sinc Oversample Factor | $F_{\rm sinc,os} $ | i_decfactor | 4096 |
| Relative Filter Length | $L_{\rm rel}$ | r_relfiltlen | 6. |
| Filter Length Scale | $\beta$ | r_beta | 0.75 |
| Weight Flag | $w_{b\rm bool}$ | i_weight | 1 |
| Weight Edge Pedestal Height | $A_{\rm sinc}$ | r_pedestal | 0. |
| Number of sinc samples | $L_{\rm sinc}$ | i_intplength | computed|
| Number of oversampled sinc samples | $N_{\rm sinc,os}$ | i_filtercoef | computed |

From these input parameters, length $L_{\rm sinc}$ of the interpolating function is calculated from 

\begin{equation}
L_{\rm sinc} = [ L_{\rm rel} /\beta ]
\end{equation}

where $[]$ denotes the nearest integer, rounding up for fractions $\ge0.5$. Then the number of coefficients computed in the highly oversampled sinc function is 

\begin{equation}
N_{\rm sinc,os} = L_{\rm sinc} F_{\rm sinc,os}
\end{equation}

An array of coefficients is calculated over this number of samples to fit one finely sampled version of a sinc function, such that any fraction can be looked up in this function and retrieve $L_{\rm sinc}$ coefficients, spaced by $F_{\rm sinc,os}$ in the array for the appropriate fraction.  This array $f_{\rm sinc,os}(s_m)$ is calculated over integers $m$ centered with 0 at the middle of the array.  Thus, we define a variable

\begin{equation}
s_m = m - m_{\rm off}  ,\quad m=0\cdots  N_{\rm sinc,os}-1
\end{equation}

where 

\begin{equation}
m_{\rm off} = \frac{N_{\rm sinc,os}-1}{2}
\end{equation}
is the offset to the center of the array.

Since the sinc can have a symmetric raised cosine taper, tapering to $A_{\rm sinc}$ at the edge, we define a weight function

\begin{equation}
W(s_m) = (1-H_{\rm sinc}) + H_{\rm sinc} \cos\left(\frac{\pi s_m}{m_{\rm off}}\right)
\end{equation}

where $H_{\rm sinc} = (1-A_{\rm sinc})/2$. When $A_{\rm sinc}=0$, $H_{\rm sinc}=0.5$, such that $W(s_0) = W(s_{N_{\rm sinc,os}-1})= 0$,  and $W(s_{m_{\rm off}}) = 1$.

The oversampled sinc function is then:

\begin{equation}
f_{\rm sinc,os}(s_m) = W(s_m) \frac{\sin \pi s_m}{\pi s_m},\quad m=0\cdots  N_{\rm sinc,os}-1
\end{equation}

Since the array is 0 indexed, in its application, there is an intrinsic delay of half the interpolating kernel length $L_{\rm sinc}/2$ that must be applied to have the interpolated output align with the input. 

In the *ampcor* implementation, the application of these coefficients $f_{\rm sinc,os}(s_m)$ is done through an intermediary array that rearranges the terms into effectively a 2-dimensional array stored as 1-dimensional, with each "row" being the $L_{\rm sinc}$ coefficitions for successive delays up to $F_{\rm sinc,os}$.

\begin{equation}
f_{\rm lookup}(n + m F_{\rm sinc,os}) = f_{\rm sinc,os}(m + n F_{\rm sinc,os}), \quad m=0\cdots F_{\rm sinc,os}-1, n=0\cdots  L_{\rm sinc}-1
\end{equation}

#### Filter application

To apply this sinc interpolator, then, it is a simple matter to translate the fraction to an index into this array, then look up the appropriate $L_{\rm sinc}$ sequential values.

Let $s$ be the real valued coordinate lying somewhere between the integer indices $p$ of some function $g(p)$, $p=0,1,2,\cdots$.  We define  the integer part of $s$ 

\begin{equation}
s_{\rm int} = \lfloor s + L_{\rm sinc}/2 \rfloor
\end{equation}

and the fractional part as 


\begin{equation}
s_{\rm frac} = s + L_{\rm sinc}/2 - s_{\rm int}
\end{equation}

Then 

\begin{equation}
g(s) = W_{\rm sinc} \sum_{k=0}^{L_{\rm sinc}-1}  g(s_{\rm int}-k) f_{\rm lookup}(k + s_{\rm frac} F_{\rm sinc,os} L_{\rm sinc} )
\end{equation}

where 

\begin{equation}
W_{\rm sinc} = \sum_{k=0}^{L_{\rm sinc}-1}  f_{\rm lookup}(k + s_{\rm frac} F_{\rm sinc,os} L_{\rm sinc} )
\end{equation}

and the $ L_{\rm sinc}/2 $ offset above accounts for the shift of the interpolating kernel in the buffers indexed from 0.



<a id="section-1.2"></a>
### 1.2 Input Data Types

*ampcor* cross-correlates two images: the reference image *refimg* and the search image *srchimg*. These images can be either real numbers or complex numbers.  

Inputs can be specified as 

real - two dimensional array of floats

complex - two dimensional array of complex numbers arranged as float (r,i) pairs

mag - *ampcor* reads a complex image but immediately detects it.


In the current implementation, all the correlations are done on detected (real-valued) images. In the first stage of correlation, the complex and mag specifications lead to the same outcome because the complex data are detected before correlation.

However, there is a second stage of correlation that oversamples the original data by a factor of 2 before correlation.  In this case the complex and mag results will be different.

In a proper implementation, real or complex correlation should be selectable, and the computation done accordingly.


<a id="section-1.3"></a>
### 1.3 Bounds checking on input

There is an extensive section of bounds and size checking of the input.  This prevents users from doing things that will likely lead to bad results.  Many of these are probably specific to this implementation.


<a id="section-1.4"></a>
### 1.4 The *ampcor* Workflow

*ampcor* performs the following steps.

1. Read in a block of lines of image 1 and image 2.
2. For the specified grid of locations to perform the correlation, detect images if complex and place detected data into buffers to correlate.  The refimg is smaller in dimensions than the srchimg, so that the srchimg can be slid around the refimg to calculate a correlation surface
3. The correlation surface is computed, along with a number of metrics, like SNR, variance in two dimensions, curvature.  The peak location is detected and used to perform a second stage of interpolation.
4. With the srchimg now centered at the estimated peak, the input data are oversampled by a factor of 2.  If the data are complex, the oversampling is done on the complex data, then detected again for correlation.  Oversampling is done by Fourier Transform.  The array is transformed, then the buffers are doubled in size and padded with zeros in the regions where the spectrum did not exist.  This needs to be done carefully because of rotations of zero frequency, depending on the kind of data it is.
5. The correlation surface is re-computed, along with a number of metrics, like SNR, variance in two dimensions, curvature.  The peak location is detected and used to perform a second stage of interpolation.
6. Then the new correlation surface is oversampled by typically a large number to allow a fine measure of the shape of the surface.  The procedure is similar to that for the complex data.
7. The peak is estimated by searching for a peak and taking the nearest point.

In [26]:
import numpy as np 
i_n2wsyi=8
i_n2wsxi=8
i_ovs=2
i_nn1=i_n2wsxi*i_ovs
i_nn2=i_n2wsyi*i_ovs
inarr = np.zeros(i_n2wsyi*i_n2wsxi,dtype=np.int)
outarr = np.zeros(i_nn1*i_nn2,dtype=np.int)
for k in range(1,i_n2wsyi//2+1):
    for l in range(1,i_n2wsxi//2+1):
        i_index = (k-1)*i_nn1 + l
        i_indexi = (k-1)*i_n2wsxi + l
        outarr[i_index-1] = 1
        inarr[i_indexi-1] = 1
        i_index = (i_nn2 - i_n2wsyi//2 + k - 1)*i_nn1 + l
        i_indexi = (k + i_n2wsyi//2 - 1)*i_n2wsxi + l
        outarr[i_index-1] = 2
        inarr[i_indexi-1] = 2
        i_index = (k-1)*i_nn1 + i_nn1 - i_n2wsxi//2 + l
        i_indexi = (k-1)*i_n2wsxi + i_n2wsxi//2 + l
        outarr[i_index-1] = 3
        inarr[i_indexi-1] = 3
        i_index = (i_nn2 - i_n2wsyi//2 + k - 1)*i_nn1 + i_nn1 - i_n2wsxi//2 + l
        i_indexi = (k + i_n2wsyi//2 - 1)*i_n2wsxi + l + i_n2wsxi//2
        outarr[i_index-1] = 4
        inarr[i_indexi-1] = 4
              
print ("in")
for k in range(0,i_n2wsyi):
    print (inarr[k*i_n2wsyi:k*i_n2wsyi+i_n2wsxi])

print ("\nout")
for k in range(0,i_nn2):
    print (outarr[k*i_nn2:k*i_nn2+i_nn1])


in
[1 1 1 1 3 3 3 3]
[1 1 1 1 3 3 3 3]
[1 1 1 1 3 3 3 3]
[1 1 1 1 3 3 3 3]
[2 2 2 2 4 4 4 4]
[2 2 2 2 4 4 4 4]
[2 2 2 2 4 4 4 4]
[2 2 2 2 4 4 4 4]

out
[1 1 1 1 0 0 0 0 0 0 0 0 3 3 3 3]
[1 1 1 1 0 0 0 0 0 0 0 0 3 3 3 3]
[1 1 1 1 0 0 0 0 0 0 0 0 3 3 3 3]
[1 1 1 1 0 0 0 0 0 0 0 0 3 3 3 3]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 2 2 2 0 0 0 0 0 0 0 0 4 4 4 4]
[2 2 2 2 0 0 0 0 0 0 0 0 4 4 4 4]
[2 2 2 2 0 0 0 0 0 0 0 0 4 4 4 4]
[2 2 2 2 0 0 0 0 0 0 0 0 4 4 4 4]
